# Instituto Tecnológico y de Estudios Superiores de Monterrey
## Maestría en Inteligencia Artificial Aplicada
### Proyecto Integrador (Gpo 10) - TC5035.10

### **Proyecto: Diseño Acelerado de Fármacos**

### Avance 2: Ingeniería de características

#### **Docentes:**
- Dra. Grettel Barceló Alonso - Profesor Titular
- Dr. Luis Eduardo Falcón Morales - Profesor Titular
- Dr. Horacio Martinez Alfaro  – Profesor Tutor

#### **Miembros del equipo:**
 - Jose Luis Artigas Arroyo - A01794906
 - José Luis Fabela Portillo - A00748551
 - Alejandro Emmanuel Silva Ortega - A01794545

In [1]:
# Importar librerias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import VarianceThreshold
import re
from pfeature import pcp_wp
from pfeature import ctd_wp
from pfeature import sep_wp
from pfeature import rri_wp
from Bio import SeqIO
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler


In [2]:
#leer csv con dataset preprocesado
df_peptidos = pd.read_csv('../data/processed/peptide_processed.csv')
df_peptidos.head(3).T

,0,1,2
Sequence,RGLRRLGRKIAHGVKKYGPTVLRIIRIA,KWKLFKKIGIGKFLHSAKKF,KWKLFKKIGPGKFLHSAKKF
Sequence_Length,28,20,20
Origin,Ovis aries,Ceropin-Magainin hybrid peptides,Ceropin-Magainin hybrid peptides
Type,Native peptide,Synthetic peptide,Synthetic peptide
Hemolytic_Activity,Human erythrocytes (4% Hemolysis=4 µM; 19% Hem...,Human erythrocytes (Hemolysis 0% =100 μM),Human erythrocytes: Hemolysis 0% (100 μM)
Cytotoxicity,NIH 3T3: IC50=16 µM,NIH-3T3 fibroblast: IC50>100 μM,NIH-3T3 fibroblast: IC50>100 μM
Binding_Target,NaN,NaN,NaN
Affinity,NaN,NaN,NaN
Mechanism,NaN,NaN,NaN
Nature,Anticancer; Antibacterial; Antifungal,Anticancer; Antibacterial,Anticancer; Antibacterial


### Eliminacion de columnas redundantes
Despues de un analis mas profundo, y una revision con el experto en el tema, se procedera a la eliminacion de columnas redundantes, ya que estos valores pueden ser obtenidos direcvtamente de la secuencia de peptidos, estas columnas son

- Chiral: Los aminoácidos L/D consisten en péptidos.
- Mass: Masa
- Formula: Formula del péptido
- Absent_amino_acids: Aminoacidos no incluidos
- Common_amino_acids: Animoacido mas comun en la sequencia
- pI: Punto isoeléctrico
- Basic_residues: Número de residuos base
- Acidic_residues: Número de residuos ácidos
- Net_charge: Carga Neta
- Polar_residues: Número de residuos polares
- Hydrophobic_residues: Número de residuos hidrofóbicos
- Hydrophobicity: Hidrofobicidad
- Boman_Index: Índice de Boman
- Half_Life: Vida media
- Aliphatic_Index: Índice alifático
- Extinction_Coefficient_cystines: Coeficiente de extinción de cistinas
- Absorbance_280nm: Absorbencia a 280nm


In [3]:
drop_cols = ['Chiral','Mass','Formula','Absent_amino_acids','Common_amino_acids','pI','Basic_residues','Acidic_residues','Net_charge','Polar_residues','Hydrophobic_residues','Hydrophobicity','Boman_Index',
             'Half_Life','Aliphatic_Index','Extinction_Coefficient_cystines','Absorbance_280nm']
df_peptidos.drop(drop_cols, axis=1, inplace=True)

In [4]:
df_peptidos.head(5).T

,0,1,2,3,4
Sequence,RGLRRLGRKIAHGVKKYGPTVLRIIRIA,KWKLFKKIGIGKFLHSAKKF,KWKLFKKIGPGKFLHSAKKF,KWKLFKKIKFLHSAKKF,KWKLFKKIPKFLHSAKKF
Sequence_Length,28,20,20,17,18
Origin,Ovis aries,Ceropin-Magainin hybrid peptides,Ceropin-Magainin hybrid peptides,Ceropin-Magainin hybrid peptides,Ceropin-Magainin hybrid peptides
Type,Native peptide,Synthetic peptide,Synthetic peptide,Synthetic peptide,Synthetic peptide
Hemolytic_Activity,Human erythrocytes (4% Hemolysis=4 µM; 19% Hem...,Human erythrocytes (Hemolysis 0% =100 μM),Human erythrocytes: Hemolysis 0% (100 μM),Human erythrocytes: Hemolysis 0% (100 μM),Human erythrocytes: Hemolysis 0% (100 μM)
Cytotoxicity,NIH 3T3: IC50=16 µM,NIH-3T3 fibroblast: IC50>100 μM,NIH-3T3 fibroblast: IC50>100 μM,NIH-3T3 fibroblast: IC50>100 μM,NIH-3T3 fibroblast: IC50>100 μM
Binding_Target,NaN,NaN,NaN,NaN,NaN
Affinity,NaN,NaN,NaN,NaN,NaN
Mechanism,NaN,NaN,NaN,NaN,NaN
Nature,Anticancer; Antibacterial; Antifungal,Anticancer; Antibacterial,Anticancer; Antibacterial,Anticancer; Antibacterial,Anticancer; Antibacterial


## Analisis de secuencias de péptidos

De acuerdo a la IUPAC[1], los aminoacidos se clasifican de acuerdo a la siguiente tabla


|symbol|Amino acid|
| :-:|:-:|
|A	| alanine|
|B	| aspartic acid or asparagine|
|C	| cysteine|
|D	| aspartic acid|
|E	| glutamic acid|
|F	| phenylalanine|
|G	| glycine|
|H	| histidine|
|I	| isoleucine|
|K	| lysine|
|L	| leucine|
|M	| methionine|
|N	| asparagine|
|P	| proline|
|Q	| glutamine|
|R	| arginine|
|S	| serine|
|T	| threonine|
|U  | selenocysteine|
|V	| valine|
|W	| tryptophan|
|X  | Desconocido u 'otro'|
|Y	| tyrosine|
|Z	| glutamic acid or glutamine|

Por lo cual, se eliminaran de la secuencia los caractares diferentes a los indicados en esta lista, de igual forma se actualizara la longitud de la secuencia para hacerla coincidir con la longitud despues de eliminar los caracteres invalidos, se eliminaran los registros con cadenas que contengan "-" y numeros (1-9) ya que estas representan peptidos modificados y contienen compuestos quimicos los cuales modifican el péptido y estos no seran considerados para el modelo.

In [5]:
df_peptidos["Sequence"]

0                RGLRRLGRKIAHGVKKYGPTVLRIIRIA
1                        KWKLFKKIGIGKFLHSAKKF
2                        KWKLFKKIGPGKFLHSAKKF
3                           KWKLFKKIKFLHSAKKF
4                          KWKLFKKIPKFLHSAKKF
                        ...                  
5846                 FLPLLAGLAANFLPKIICKIARKC
5847                  RIIDRLWLVRRPOKPKFVLVWVL
5848                               FHAVPQSFYT
5849                            KLWCKSSQVPQSR
5850    TKEQKEQIAKATGLTTKQVRNWYVQLNASIKVCMCSC
Name: Sequence, Length: 5851, dtype: object

In [6]:
df_peptidos["Sequence"].map(lambda x: re.sub(r"[\"]",'',x))

0                RGLRRLGRKIAHGVKKYGPTVLRIIRIA
1                        KWKLFKKIGIGKFLHSAKKF
2                        KWKLFKKIGPGKFLHSAKKF
3                           KWKLFKKIKFLHSAKKF
4                          KWKLFKKIPKFLHSAKKF
                        ...                  
5846                 FLPLLAGLAANFLPKIICKIARKC
5847                  RIIDRLWLVRRPOKPKFVLVWVL
5848                               FHAVPQSFYT
5849                            KLWCKSSQVPQSR
5850    TKEQKEQIAKATGLTTKQVRNWYVQLNASIKVCMCSC
Name: Sequence, Length: 5851, dtype: object

In [7]:
# Crear un diccionario para contar la aparición de cada carácter de la secuencia de aminoácidos
char_count = {}

# Iterar sobre cada secuencia en la columna 'Sequence'
for sequence in df_peptidos['Sequence']:
    # Convertir la secuencia en un conjunto de caracteres únicos
    unique_chars = set(sequence)
    # Contar la aparición de cada carácter en el conjunto
    for char in unique_chars:
        if char in char_count:
            char_count[char] += 1
        else:
            char_count[char] = 1

# Convertir el diccionario en un DataFrame para una mejor visualización
char_count_df = pd.DataFrame(list(char_count.items()), columns=['Character', 'Count'])
char_count_df['%'] = (char_count_df['Count'] / df_peptidos.shape[0]).round(4) *100

# Número de péptidos por caracter
char_count_df.T

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
Character,A,G,I,L,V,P,R,K,H,T,...,Φ,(,),],[,b,6,2,1,7
Count,3014,2964,3081,3858,2932,2435,3207,3390,1033,1684,...,11,4,4,1,1,1,1,1,1,1
%,51.51,50.66,52.66,65.94,50.11,41.62,54.81,57.94,17.66,28.78,...,0.19,0.07,0.07,0.02,0.02,0.02,0.02,0.02,0.02,0.02


Es importante identificar los caracteres contenidos en la secuencia de aminoácidos de los péptidos, por tal motivo vamos a analizar su ocurrencia en el total de péptidos. La lista utilizada es la que se describio anteriormente (IUPAC), sin embargo, para obtener las características con la librería de Pfeatures no se reconocen 4 de estos: B, U, X, Z. Asímismo existen otros caracteres no incluidos en IUPAC por lo que es necesario analizar que tan frecuentes son.

In [8]:
# Aminoácidos Pfeature
aa_pfeature = ['A','C','D','E','F','G','H','I','K','L','M','N','P',
              'Q','R','S','T','V','W','Y']

# Aminoácidos IUPAC - Pfeatures
aa_iupac = ['B','U','X','Z']

# Aminoácidos en minúsculas
aa_min = [elemento.lower() for elemento in (aa_pfeature + aa_iupac)]

print ('Total de aminoácidos IUPAC:', len(aa_iupac) + len(aa_pfeature))
print ('Total de aminoácidos Pfeatures:', len(aa_pfeature))
print ('Total de aminoácidos que están en UIPAC pero no reconoce Pfeatures:', len(aa_iupac))



Total de aminoácidos IUPAC: 24
Total de aminoácidos Pfeatures: 20
Total de aminoácidos que están en UIPAC pero no reconoce Pfeatures: 4


In [9]:
# Crear un diccionario para contar la aparición de cada carácter
char_count = {}

# Iterar sobre cada secuencia en la columna 'Sequence'
for sequence in df_peptidos['Sequence']:
    # Convertir la secuencia en un conjunto de caracteres únicos
    unique_chars = set(sequence)
    # Contar la aparición de cada carácter en el conjunto
    for char in unique_chars:
        if char in char_count:
            char_count[char] += 1
        else:
            char_count[char] = 1

# Convertir el diccionario en un DataFrame para una mejor visualización
char_count_df = pd.DataFrame(list(char_count.items()), columns=['Character', 'Count'])
char_count_df['%'] = (char_count_df['Count'] / df_peptidos.shape[0]).round(4) *100


print('Total de caracteres únicos: ', len(char_count_df))

Total de caracteres únicos:  64


In [10]:
# Aminoácidos Pfeatures Mayúsculas
char_count_aapf = char_count_df[char_count_df['Character'].isin(aa_pfeature)]
print('Total de caracteres de Pfeatures incluidos en el dataset: ', len(char_count_aapf))
char_count_aapf.sort_values(by='Count', ascending=False).T

Total de caracteres de Pfeatures incluidos en el dataset:  20


,3,7,6,2,0,1,4,12,5,11,13,10,17,14,15,16,9,19,18,8
Character,L,K,R,I,A,G,V,F,P,W,S,Y,N,D,E,Q,T,M,C,H
Count,3858,3390,3207,3081,3014,2964,2932,2875,2435,2143,2077,1985,1942,1825,1824,1808,1684,1202,1116,1033
%,65.94,57.94,54.81,52.66,51.51,50.66,50.11,49.14,41.62,36.63,35.5,33.93,33.19,31.19,31.17,30.9,28.78,20.54,19.07,17.66


In [11]:
# Aminoácidos UIPAC no incluídos en Pfeatures Mayúsculas
char_count_aauipac = char_count_df[char_count_df['Character'].isin(aa_iupac)]
print('Total de caracteres de Pfeatures incluidos en el dataset: ', len(char_count_aauipac))
char_count_aauipac.sort_values(by='Count', ascending=False).T

Total de caracteres de Pfeatures incluidos en el dataset:  4


,29,51,37,41
Character,X,Z,B,U
Count,691,364,31,5
%,11.81,6.22,0.53,0.09


La ocurrencia de Z, B, U es muy baja por lo que se procederá a eliminar todos los péptidos que las contienen. 
Para el caso de los péptidos con 'x' se re-escribiran aquellas que solo tengan X al inicio o al final. Por ejemplo, si la secuencia es XXVKKGLISY reescribirla como VKKGLISY, si la secuencia es KKGLISYXXX entonces se reescribirá como KKGLISY. Sin embargo, si la secuencia es KKGXXLISY entonces se eliminará la secuencia del dataset.

In [12]:
# Aminoácidos UIPAC no incluídos en Pfeatures Mayúsculas
char_count_aamin = char_count_df[char_count_df['Character'].isin(aa_min)]
print('Total de caracteres de Pfeatures incluidos en el dataset: ', len(char_count_aamin))
char_count_aamin.sort_values(by='Count', ascending=False).T

Total de caracteres de Pfeatures incluidos en el dataset:  24


,30,23,25,20,39,21,28,31,27,34,...,47,26,48,43,33,49,24,44,53,59
Character,r,l,k,f,x,a,v,w,i,n,...,c,g,q,e,m,h,d,u,z,b
Count,141,132,112,78,77,54,54,52,44,33,...,21,19,15,13,11,8,7,1,1,1
%,2.41,2.26,1.91,1.33,1.32,0.92,0.92,0.89,0.75,0.56,...,0.36,0.32,0.26,0.22,0.19,0.14,0.12,0.02,0.02,0.02


La distincion entre los tipos de aminoácidos L o sinteticos y D o naturales se puede ver en estos resultados, siendo representados los tipo D con letras mayusculuas y los tipo L con minusculas. Gracias a este analisis podemos mostrar que no es relevante mantener la distincion entre estos aminoácidos y se hara un tratamiento a estos datos para convertir los caracteres en minusculas a mayusculas mas adelante.  

In [13]:
# Número de péptidos en los que está cada caracter que no es aminoácido IUPAC:
char_count_otros = char_count_df[~char_count_df['Character'].isin(aa_pfeature + aa_iupac + aa_min)]
print('Total de caracteres no incluidos en el UIPAC: ', len(char_count_otros))
char_count_otros.sort_values(by='Count', ascending=False).T

Total de caracteres no incluidos en el UIPAC:  16


,46,52,54,32,55,56,45,50,42,38,57,58,60,61,62,63
Character,Ⓧ,",",Φ,O,(,),-,j,,工,],[,6,2,1,7
Count,236,18,11,10,4,4,3,3,2,1,1,1,1,1,1,1
%,4.03,0.31,0.19,0.17,0.07,0.07,0.05,0.05,0.03,0.02,0.02,0.02,0.02,0.02,0.02,0.02


In [14]:
char_count_otros['Character'].unique()

array(['O', '工', '\xa0', '-', 'Ⓧ', 'j', ',', 'Φ', '(', ')', ']', '[', '6',
       '2', '1', '7'], dtype=object)

La frecuencia de los caractéres no incluidos en IUPAC es muy baja, por lo que también se eliminarán los péptidos que las contienen.

In [15]:
# Ajuste de las secuencias de aminoácidos
filtered_df = df_peptidos.copy()
# 1. Poner en mayúsculas todos los aminoácidos de las secuencias
filtered_df['Sequence'] = filtered_df['Sequence'].str.upper()
# 2. Identificar los péptidos con valores de X
aa_x = filtered_df[filtered_df['Sequence'].apply(lambda x: 'X' in x)]
# 3. Conservar solo los péptidos que contengan los aminoácidos para pfeatures
filtered_df = filtered_df[filtered_df['Sequence'].apply(lambda x: all(char in aa_pfeature for char in x))]

In [16]:
# Identifica todos péptidos con valores de X
total_x = len(aa_x)
print('Total de péptidos con X', total_x)

# Inician con X 
inicio_x = aa_x[aa_x['Sequence'].str.startswith('X')]
print('Péptidos que empiezan con X: ', len(inicio_x))
# Función para quitar los caracteres 'x' al inicio de la secuencia
def quitar_x_inicio(secuencia):
    return re.sub(r'^X+', '', secuencia)
# Aplicar la función a la columna 'Sequence'
aa_x['Sequence'] = aa_x['Sequence'].apply(quitar_x_inicio)

# Finalizan con X
fin_x = aa_x[aa_x['Sequence'].str.endswith('X')].copy()
print('Péptidos que finalizan con X: ', len(fin_x))
# Función para quitar los caracteres 'x' al final de la secuencia
def quitar_x_final(secuencia):
    return re.sub(r'X+$', '', secuencia)
# Aplicar la función a la columna 'Sequence'
aa_x['Sequence'] = aa_x['Sequence'].apply(quitar_x_final)

# Elimina los péptidos con X intermedia
aa_x = aa_x[aa_x['Sequence'].apply(lambda x: all(char in aa_pfeature for char in x))]
print('Péptidos con X intermedia:', total_x - len(aa_x))
print('Peptidos conservados: ', len(aa_x))

Total de péptidos con X 727
Péptidos que empiezan con X:  257
Péptidos que finalizan con X:  177
Péptidos con X intermedia: 551
Peptidos conservados:  176


In [17]:
# Agrega los peptidos con X modificados al dataframe de péptidos depurados
filtered_df = pd.concat([filtered_df, aa_x], ignore_index=True)

print('Total de péptidos: ', df_peptidos.shape[0])
print('Péptidos solo con aminoácidos para pFeatures: ', filtered_df.shape[0])
print('Péptidos eliminados: ', round((df_peptidos.shape[0]-filtered_df.shape[0])/df_peptidos.shape[0],4) * 100, '%')

Total de péptidos:  5851
Péptidos solo con aminoácidos para pFeatures:  4679
Péptidos eliminados:  20.03 %


In [18]:
# Valida los aminoácidos contenidos
char_count = {}

# Iterar sobre cada secuencia en la columna 'Sequence'
for sequence in filtered_df['Sequence']:
    # Convertir la secuencia en un conjunto de caracteres únicos
    unique_chars = set(sequence)
    # Contar la aparición de cada carácter en el conjunto
    for char in unique_chars:
        if char in char_count:
            char_count[char] += 1
        else:
            char_count[char] = 1

# Convertir el diccionario en un DataFrame para una mejor visualización
char_count_df = pd.DataFrame(list(char_count.items()), columns=['Character', 'Count'])
char_count_df['%'] = (char_count_df['Count'] / filtered_df.shape[0]).round(4) *100


print('Total de caracteres únicos: ', len(char_count_df))

Total de caracteres únicos:  20


In [19]:
filtered_df.to_csv('../data/processed/peptide_filtered.csv', index = False)

### Convertir dataframe to FASTA 

Para los siguientes pasos del proceso, es necesario hacer la conversion de los datos filtrados a formato FASTA, este formato es el estandard usado para las secuencias de aminoácidos

In [20]:

with open('../data/processed/VEGF.fasta','w', encoding='UTF8') as f:
    for _, row in filtered_df.iterrows():
        f.write(f">{row['VEGF']}\n{row['Sequence']}\n")
    


### Ejecutar CD-HIT

CD-HIT es un programa ampliamente utilizado para agrupar secuencias biológicas con el fin de reducir la redundancia de secuencias y mejorar el rendimiento de otros análisis de secuencias. En respuesta al rápido aumento de la cantidad de datos de secuenciación producidos por las tecnologías de secuenciación de próxima generación, hemos desarrollado un nuevo programa CD-HIT acelerado con una novedosa estrategia de paralelización y algunas otras técnicas para permitir la agrupación eficiente de dichos conjuntos de datos. Nuestras pruebas demostraron una aceleración muy buena derivada de la paralelización para hasta ∼24 núcleos y una aceleración cuasi-lineal para hasta ∼8 núcleos. El CD-HIT mejorado es capaz de manejar conjuntos de datos muy grandes en mucho menos tiempo que las versiones anteriores[2].

In [21]:
#Ejecutar cd-hit en colab, para windows usar el comando con wsl
#! cd-hit -i data/processed/VEGF.fasta -o data/processed/VEGF.txt -c 0.99

import subprocess

# Define the WSL command buscar 
command = ["wsl", "cd-hit", "-i", "../data/processed/VEGF.fasta", "-o", "../data/processed/VEGF.txt", "-c", "0.99"]

# Run the command
result = subprocess.run(command, capture_output=True, text=True)

# Print the output and errors
print("STDOUT:", result.stdout)
print("STDERR:", result.stderr)


STDOUT: ================================================================
Program: CD-HIT, V4.8.1 (+OpenMP), Aug 20 2021, 08:39:56
Command: cd-hit -i ../data/processed/VEGF.fasta -o
         ../data/processed/VEGF.txt -c 0.99

Started: Sun Feb  2 18:14:55 2025
                            Output                              
----------------------------------------------------------------
total seq: 3276
longest and shortest : 97 and 11
Total letters: 68527
Sequences have been sorted

Approximated minimal memory consumption:
Sequence        : 0M
Buffer          : 1 X 10M = 10M
Table           : 1 X 65M = 65M
Miscellaneous   : 0M
Total           : 76M

Table limit with the given memory limit:
Max number of representatives: 4000000
Max number of word counting entries: 90445565


comparing sequences from          0  to       3276
...
     3276  finished       2704  clusters

Approximated maximum memory consumption: 76M
writing new database
writing clustering information
program completed !


# Funciones de Pfeature

La librería pfeature es una herramienta de Python utilizada para extraer y analizar características de secuencias proteicas y nucleotídicas. Proporciona métodos para calcular descriptores físicos, químicos, estadísticos y estructurales de biomoléculas, lo que la hace útil en bioinformática, predicción de propiedades funcionales y clasificación de secuencias en aprendizaje automático.


## Aplicaciones en Bioinformática
pfeature se usa ampliamente en:
- Predicción de propiedades de proteínas (hidrofobicidad, accesibilidad al solvente, estabilidad).
- Clasificación de proteínas en modelos de Machine Learning.
- Análisis de secuencias en estudios evolutivos.
- Desarrollo de modelos para predicción de interacciones proteína-ligando.

pfeature es una poderosa herramienta en bioinformática para analizar secuencias biológicas de manera rápida y eficiente. Su capacidad de generar múltiples descriptores permite su aplicación en aprendizaje automático, descubrimiento de fármacos y análisis estructural de proteínas[3].


In [22]:
#Lee el archivo creado por CD-HIT y almacena las etiquetas en un archivo CSV para su uso posterior
cd_hit_path = "../data/processed/VEGF.txt"
records_cd_hit = list(SeqIO.parse(cd_hit_path, "fasta"))
with open('../data/processed/VEGF_y.csv','w', encoding='UTF8') as f:
    for record in records_cd_hit:
        f.write(f'{record.id} \n')

### Physico-Chemical Properties Composition: 30 Descriptores

Este modulo calcula la fracción de cada propiedas físico-química de una secuencia. Las siguientes propiedades se han incorporado a Pfeature para calcular caracteristicas composicionales basadas en propiedades físico-químicas estandard.

- Positively Charged → Carga Positiva
- Negatively Charged → Carga Negativa
- Neutral Charge → Carga Neutra
- Polarity in residues → Polaridad en residuos
- Non-polarity in residues → No polaridad en residuos
- Aliphaticity → Alifaticidad
- Cyclicity → Ciclicidad
- Aromaticity → Aromaticidad
- Acidity → Acidez
- Basicity → Basicidad
- Neutral (pH) → Neutro (pH)
- Hydrophobicity → Hidrofobicidad
- Hydrophilicity → Hidrofilicidad
- Neutral towards water → Neutro frente al agua
- Hydroxylic → Hidroxílico
- Sulphur Content → Contenido de Azufre
- Tiny → Pequeño
- Small → Mediano
- Large → Grande

parfa el cálculo de estas propiedades se usa la siguiente formula 

### 𝐶𝑃𝑖 = 𝑃𝑖/𝐿 

donde

- 𝐶𝑃𝑖 = Propiedades físico-quimicas de la composicion de tipo 𝑖.
- 𝑃𝑖 = suma de la propiedad de tipo 𝑖.
- 𝐿 = Longitud de la secuencia. 

In [23]:
# Physico-Chemical Properties Composition (PCP)

def pcp(input):
  a = input.rstrip('txt')
  output = a + 'dpc.csv'
  df_out = pcp_wp(input, output)
  df_in = pd.read_csv(output)
  return df_in

feature_pcp = pcp(cd_hit_path)
feature_pcp

,PCP_PC,PCP_NC,PCP_NE,PCP_PO,PCP_NP,PCP_AL,PCP_CY,PCP_AR,PCP_AC,PCP_BS,...,PCP_SA_EX,PCP_SA_IN,PCP_TN,PCP_SM,PCP_LR,PCP_Z1,PCP_Z2,PCP_Z3,PCP_Z4,PCP_Z5
0,0.357,0.000,0.643,0.071,0.571,0.571,0.036,0.036,0.000,0.357,...,0.143,0.143,0.214,0.357,0.643,0.075,-0.601,-1.386,0.315,0.169
1,0.400,0.000,0.600,0.050,0.550,0.350,0.000,0.200,0.000,0.400,...,0.350,0.100,0.200,0.200,0.800,-0.407,0.018,-0.867,0.655,0.111
2,0.400,0.000,0.600,0.050,0.550,0.350,0.050,0.200,0.000,0.400,...,0.400,0.150,0.200,0.250,0.750,-0.295,0.118,-0.689,0.732,0.198
3,0.444,0.000,0.556,0.056,0.500,0.278,0.056,0.222,0.000,0.444,...,0.444,0.167,0.111,0.167,0.833,-0.556,0.583,-0.806,0.904,0.262
4,0.125,0.062,0.812,0.062,0.750,0.688,0.000,0.062,0.062,0.125,...,0.188,0.062,0.312,0.500,0.500,-0.768,-1.499,-0.647,-0.537,0.250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2699,0.400,0.000,0.600,0.280,0.320,0.240,0.000,0.080,0.000,0.400,...,0.000,0.240,0.520,0.520,0.480,1.878,-0.005,-0.327,0.854,-0.409
2700,0.412,0.000,0.588,0.176,0.412,0.353,0.000,0.059,0.000,0.412,...,0.000,0.118,0.529,0.529,0.471,1.835,-0.429,-0.168,1.029,-0.636
2701,0.444,0.000,0.556,0.167,0.389,0.333,0.111,0.056,0.000,0.444,...,0.333,0.111,0.222,0.333,0.667,1.354,0.284,-1.157,0.621,0.075
2702,0.909,0.091,0.000,0.000,0.000,0.000,0.000,0.000,0.091,0.909,...,0.182,0.000,0.000,0.000,1.000,3.371,2.150,-3.100,1.487,-0.134


Correlacion: Descarta características redundantes con alta correlación entre sí.

In [24]:
#correlacion
corr_matrix = pd.DataFrame(feature_pcp).corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.9)]
corr_matrix_pcp = feature_pcp.drop(to_drop, axis=1)
corr_matrix_pcp.to_csv('../data/processed/corr_matrix_pcp.csv', index = False)
corr_matrix_pcp

,PCP_PC,PCP_NC,PCP_PO,PCP_NP,PCP_AL,PCP_CY,PCP_AR,PCP_HB,PCP_HL,PCP_NT,...,PCP_SA_BU,PCP_SA_EX,PCP_SA_IN,PCP_TN,PCP_SM,PCP_Z1,PCP_Z2,PCP_Z3,PCP_Z4,PCP_Z5
0,0.357,0.000,0.071,0.571,0.571,0.036,0.036,0.464,0.393,0.179,...,0.536,0.143,0.143,0.214,0.357,0.075,-0.601,-1.386,0.315,0.169
1,0.400,0.000,0.050,0.550,0.350,0.000,0.200,0.450,0.400,0.150,...,0.550,0.350,0.100,0.200,0.200,-0.407,0.018,-0.867,0.655,0.111
2,0.400,0.000,0.050,0.550,0.350,0.050,0.200,0.450,0.450,0.150,...,0.500,0.400,0.150,0.200,0.250,-0.295,0.118,-0.689,0.732,0.198
3,0.444,0.000,0.056,0.500,0.278,0.056,0.222,0.500,0.500,0.056,...,0.444,0.444,0.167,0.111,0.167,-0.556,0.583,-0.806,0.904,0.262
4,0.125,0.062,0.062,0.750,0.688,0.000,0.062,0.562,0.125,0.312,...,0.750,0.188,0.062,0.312,0.500,-0.768,-1.499,-0.647,-0.537,0.250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2699,0.400,0.000,0.280,0.320,0.240,0.000,0.080,0.240,0.400,0.360,...,0.440,0.000,0.240,0.520,0.520,1.878,-0.005,-0.327,0.854,-0.409
2700,0.412,0.000,0.176,0.412,0.353,0.000,0.059,0.294,0.412,0.294,...,0.588,0.000,0.118,0.529,0.529,1.835,-0.429,-0.168,1.029,-0.636
2701,0.444,0.000,0.167,0.389,0.333,0.111,0.056,0.278,0.556,0.278,...,0.333,0.333,0.111,0.222,0.333,1.354,0.284,-1.157,0.621,0.075
2702,0.909,0.091,0.000,0.000,0.000,0.000,0.000,0.000,0.909,0.091,...,0.000,0.182,0.000,0.000,0.000,3.371,2.150,-3.100,1.487,-0.134


### Composition enhanced Transition and Distribution: 189 descriptors [4]

Composition-Enhanced Transition and Distribution (CTD) es un enfoque utilizado en bioinformática y química computacional para describir secuencias de aminoácidos o nucleótidos. Se basa en tres principios clave:

Composición (Composition): Representa la proporción de cada tipo de aminoácido o nucleótido en una secuencia.
Transición (Transition): Describe la frecuencia con la que un tipo de aminoácido/nucleótido cambia a otro a lo largo de la secuencia.
Distribución (Distribution): Indica la posición relativa de ciertos tipos de aminoácidos/nucleótidos dentro de la secuencia.

Cómo funciona CTD en la descripción de secuencias

1. Composición (C - Group 1) 

- Calcula el porcentaje de cada grupo de aminoácidos en la secuencia.
- Se agrupan los aminoácidos según ciertas propiedades fisicoquímicas, como hidrofobicidad o carga.

2. Transición (T - Group 2)

- Representa la frecuencia de cambio entre dos grupos de aminoácidos consecutivos en la secuencia.
- Se calcula como el número de transiciones entre los grupos dividido por el total de transiciones posibles.

3. Distribución (D - Group 3)

- Indica en qué posiciones de la secuencia aparecen los primeros, el 25%, 50%, 75% y 100% de los residuos de un grupo específico.

|Attributes|Group1|Group 2|Group 3|
|:-:|:-:|:-:|:-:|
|Hydrophobicity| R,K,E,D,Q,N| G,A,S,T,P,H,Y| C,L,V,I,M,F,W|
|Normalized Vander Waals volume| G,A,S,T,P,D| N,V,E,Q,I,L| M,H,K,F,R,Y,W|
|Polarity| L,I,F,W,C,M,V,Y| P,A,T,G,S| H,Q,R,K,N,E,D|
|Polarizability| G,A,S,D,T| C,P,N,V,E,Q,I,L| K,M,H,F,R,Y,W|
|Charge| K,R| A,N,C,Q,G,H,I,L,M,F,P,S,T,W,Y,V| D,E|
|Secondary structure| E,A,L,M,Q,K,R,H| V,I,Y,C,W,F,T| G,N,P,S,D|
|Solvent accessibility| A,L,F,C,G,I,V,W| R,K,Q,E,N,D| M,S P,T,H,Y|

Los atributos de CTD permiten capturar información estructural y funcional de secuencias biológicas, convirtiéndolas en vectores numéricos que pueden ser utilizados para la predicción y clasificación en bioinformática.

In [25]:
# Composition enhanced Transition and Distribution: (CTD) 

def ctd(input):
  a = input.rstrip('txt')
  output = a + 'ctd.csv'
  df_out = ctd_wp(input, output)
  df_in = pd.read_csv(output)
  return df_in

feature_ctd = ctd(cd_hit_path)
feature_ctd

,CeTD_HB1,CeTD_HB2,CeTD_HB3,CeTD_VW1,CeTD_VW2,CeTD_VW3,CeTD_PO1,CeTD_PO2,CeTD_PO3,CeTD_PZ1,...,CeTD_0_p_SS3,CeTD_25_p_SS3,CeTD_50_p_SS3,CeTD_75_p_SS3,CeTD_100_p_SS3,CeTD_0_p_SA3,CeTD_25_p_SA3,CeTD_50_p_SA3,CeTD_75_p_SA3,CeTD_100_p_SA3
0,32.14,35.71,32.14,28.57,32.14,39.29,35.71,28.57,35.71,25.00,...,0,2,4,6,9,0,1,2,3,4
1,35.00,25.00,40.00,20.00,20.00,60.00,40.00,20.00,40.00,20.00,...,0,1,3,5,7,0,0,1,1,2
2,35.00,30.00,35.00,25.00,15.00,60.00,35.00,25.00,40.00,20.00,...,0,1,3,5,7,0,0,1,2,3
3,38.89,22.22,38.89,16.67,16.67,66.67,38.89,16.67,44.44,11.11,...,0,1,3,5,7,0,0,1,2,3
4,18.75,31.25,50.00,37.50,43.75,18.75,50.00,31.25,18.75,37.50,...,0,0,1,2,3,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2699,32.00,48.00,20.00,45.45,0.00,54.55,20.00,40.00,40.00,40.00,...,0,2,4,6,8,0,1,3,4,6
2700,29.41,47.06,23.53,42.86,0.00,57.14,23.53,35.29,41.18,35.29,...,0,1,2,3,5,0,0,1,1,2
2701,55.56,27.78,16.67,29.41,17.65,52.94,16.67,27.78,55.56,16.67,...,0,2,5,7,10,0,0,1,1,2
2702,100.00,0.00,0.00,0.00,9.09,90.91,0.00,0.00,100.00,0.00,...,0,2,5,8,11,0,0,0,0,0


Se aplicara la transformación de PCA a las caracteristicas de CTD para reducir su dimensionalidad, manteniendo la mayor varianza posible.

In [26]:
def aplicar_pca(data, threshold = 0.90):
    print("Aplicando operación: PCA")
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(data)
    # Aplicar PCA
    pca = PCA().fit(X_scaled)
    # Calcular la varianza explicada acumulada
    explained_variance = np.cumsum(pca.explained_variance_ratio_)
    num_components = np.argmax(explained_variance >= threshold) + 1
    pca = PCA(n_components=num_components)  # Número de componentes principales
    X_pca = pca.fit_transform(X_scaled)
    pca_columns = [f'PC{i+1}' for i in range(X_pca.shape[1])]  # Nombrar los componentes como PC1, PC2, etc.
    df_pca = pd.DataFrame(X_pca, columns=pca_columns)
    return df_pca

In [27]:
pca_ctd = aplicar_pca(feature_ctd)
pca_ctd.to_csv('../data/processed/pca_ctd.csv', index = False)
pca_ctd

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,PC12
0,4.465813,2.238494,1.310017,-2.356987,1.922528,-0.451939,0.978216,-1.303886,-0.123767,0.780396,-1.447275,-0.078434
1,-1.828824,3.707624,0.998182,-3.455248,1.169127,-0.739495,-0.578227,-0.117767,0.353232,-2.316752,-0.423856,0.288518
2,-2.042295,3.572486,-1.106014,-3.093488,0.903204,-1.017569,0.073823,-0.732423,1.842737,-2.697651,0.063404,0.224664
3,-3.946575,5.021183,0.124242,-3.025322,-0.214241,-1.147517,0.571092,0.412455,0.890656,-2.336612,-0.075183,0.033389
4,-3.182656,-6.694143,2.874705,0.816326,0.862016,0.666197,-2.222052,-0.948383,-0.788840,-0.248492,0.159610,0.657820
...,...,...,...,...,...,...,...,...,...,...,...,...
2699,2.191096,0.310381,-8.644810,-3.752692,1.559313,5.157465,-1.564594,-0.115044,-0.282376,-0.721614,-0.455279,2.394572
2700,-3.039970,-0.641169,-4.636086,-2.392446,2.341733,-1.132851,-1.104915,0.720910,-1.847882,-0.568111,0.674691,1.603461
2701,-4.126764,4.551046,-4.846577,1.338521,-0.069354,4.393361,0.463382,-1.405499,-1.271860,0.961104,0.422888,1.082130
2702,-10.472682,12.110773,-6.206620,2.676183,-1.859294,6.627050,-1.970334,-1.877130,-1.304498,0.329669,-0.330525,-1.962865


# Entropia de Shannon

La entropía de Shannon es una medida de la incertidumbre o diversidad en una secuencia de símbolos. En el contexto de los aminoácidos, se usa para cuantificar la variabilidad en la composición de una secuencia de proteínas o en una alineación de secuencias múltiples.

### Fórmula de la Entropía de Shannon
La entropía 𝐻 se define como:

![alt text](image-1.png)

​
Donde:
𝑝𝑖 es la frecuencia de aparición de un aminoácido 𝑖 en la secuencia.
𝑛 es el número total de tipos de aminoácidos (generalmente 20).
El logaritmo se toma en base 2 para obtener la entropía en bits.

#Interpretación en Secuencias de Proteínas

- Alta entropía: La secuencia tiene una composición muy diversa, sin un aminoácido predominante. Indica mayor variabilidad y menos conservación.
- Baja entropía: Algunos aminoácidos dominan la secuencia, lo que sugiere regiones conservadas estructural o funcionalmente.

### Aplicaciones en Bioinformática

1. Análisis de Conservación: Se usa en alineamientos múltiples para identificar regiones conservadas en familias de proteínas.
2. Predicción de Estructura y Función: Regiones con baja entropía suelen ser funcionalmente importantes (ej., sitios activos, dominios de unión).
3. Estudios Evolutivos: Ayuda a detectar señales de selección natural en proteínas.


In [28]:
def sep(input): #sShannon entropy
  a = input.rstrip('txt')
  output = a + 'sep.csv'
  df_out = sep_wp(input, output)
  df_in = pd.read_csv(output)
  return df_in

feature_sep = sep(cd_hit_path)
feature_sep.to_csv('../data/processed/Shannon_entropy.csv', index = False)
feature_sep

,SEP
0,3.200
1,2.802
2,2.902
3,2.703
4,3.031
...,...
2699,2.549
2700,2.324
2701,2.711
2702,0.866


# Repetitive Residue Information (RRI)

Repetitive Residue Information (RRI) es una métrica utilizada en bioinformática para analizar la redundancia y patrones de repetición en la secuencia de aminoácidos de péptidos y proteínas. Evalúa cuán frecuente es la aparición de residuos específicos en una secuencia y cómo esta repetición puede influir en la estructura y función de la biomolécula.

### Definición de RRI
RRI cuantifica la presencia de residuos repetitivos dentro de una secuencia peptídica. Se basa en la frecuencia de aparición de cada aminoácido y su distribución dentro de la secuencia. Esto permite identificar si una proteína o péptido tiene regiones altamente conservadas o patrones estructurales específicos, como repeticiones en hélices alfa o láminas beta.

La fórmula general para calcular RRI es:

![alt text](image.png)

donde:
- 𝑓𝑖 es la frecuencia del aminoácido 𝑖 en la secuencia.
- 𝑁 es la longitud total de la secuencia.

Esta ecuación es similar a la entropía de Shannon, midiendo la diversidad de residuos en la secuencia.

### Aplicaciones de RRI en Bioinformática

1. Identificación de motivos estructurales y funcionales

- Péptidos con residuos repetitivos pueden formar estructuras secundarias estables, como hélices alfa anfipáticas o láminas beta repetitivas.

2. Predicción de propiedades fisicoquímicas

- Secuencias con alta repetición pueden afectar la hidrofobicidad, la solubilidad y la interacción con membranas celulares.

3. Diseño de péptidos terapéuticos

- Se utiliza en el diseño de péptidos antimicrobianos y en la optimización de fármacos basados en péptidos, evitando redundancias que puedan generar respuestas inmunogénicas no deseadas.

4. Análisis evolutivo

- Ayuda a estudiar la conservación de secuencias en proteínas homólogas y en el desarrollo de alineamientos múltiples.

El RRI es una métrica valiosa para analizar la redundancia de residuos en secuencias peptídicas. Su aplicación en bioinformática estructural, diseño de péptidos y análisis evolutivo permite mejorar la comprensión de la relación entre la secuencia y la función de las proteínas.

In [29]:
def rri(input):
  a = input.rstrip('txt')
  output = a + 'rri.csv'
  df_out = rri_wp(input, output)
  df_in = pd.read_csv(output)
  return df_in

feature_rri = rri(cd_hit_path)
feature_rri.to_csv('../data/processed/feature_rri.csv', index = False)
feature_rri

,RRI_A,RRI_C,RRI_D,RRI_E,RRI_F,RRI_G,RRI_H,RRI_I,RRI_K,RRI_L,RRI_M,RRI_N,RRI_P,RRI_Q,RRI_R,RRI_S,RRI_T,RRI_V,RRI_W,RRI_Y
0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.5,1.67,1.0,0.0,0.0,1.0,0.0,1.33,0.0,1.0,1.0,0.0,1.0
1,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.57,1.0,0.0,0.0,0.0,0.0,0.00,1.0,0.0,0.0,1.0,0.0
2,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.57,1.0,0.0,0.0,1.0,0.0,0.00,1.0,0.0,0.0,1.0,0.0
3,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.57,1.0,0.0,0.0,1.0,0.0,0.00,1.0,0.0,0.0,1.0,0.0
4,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,2.00,1.0,0.0,0.0,0.0,0.0,0.00,1.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2699,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,8.00,1.0,0.0,0.0,1.0,0.0
2700,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,1.40,0.0,0.0,0.0,1.0,0.0
2701,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,2.00,1.0,0.0,0.0,2.0,1.0,2.33,0.0,0.0,0.0,0.0,0.0
2702,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.00,0.0,0.0,0.0,0.0,0.0,9.00,0.0,0.0,0.0,0.0,0.0


### Conclusiones

### Referencias

1. INTERNATIONAL UNION OF PURE AND APPLIED CHEMISTRY (IUPAC). https://iupac.qmul.ac.uk/AminoAcid/A2021.html#AA20
2. Fu, L., Niu, B., Zhu, Z., Wu, S., & Li, W. (2012). CD-HIT: accelerated for clustering the next-generation sequencing data. Bioinformatics (Oxford, England), 28(23), 3150–3152. https://doi.org/10.1093/bioinformatics/bts565
3. Raghava, S. P. a. D. a. P. D. K. N. S. G. M. R. K. P. a. V. P. a. L. H. K. S. J. P. G. (n.d.). Pfeature: A web server for computing protein and peptide features. https://webs.iiitd.edu.in/raghava/pfeature/
4. Dubchak I, et al. Proceedings of the National Academy of Sciences of the United States of America